In [4]:
import pandas as pd
import bz2
import json

with bz2.open('./pair_task/heldout_pair_data.jsonlist.bz2', 'rt') as f:
    data = [json.loads(line) for line in f]

og_df = pd.DataFrame(data)

In [17]:
len(data)

807

In [5]:
# RANDOM SAMPLE OF 500 
sample_df = og_df.sample(n=500, random_state=12).reset_index(drop=True)

In [6]:
sample_df.head()

,op_author,op_text,op_title,positive,negative,op_name
0,whyhavename,(Edit: My view has been change though there is...,CMV: People shouldn't change gender,"{'ancestor': 't1_cswzpc1', 'author': 'Mqzp', '...","{'ancestor': 't1_csz58e3', 'author': '[deleted...",t3_3cm6jy
1,WumboWombo,"I'm young, so of course my biggest concern at ...",CMV: America's economy is destined to fail,"{'ancestor': 't1_cr2suj3', 'author': 'huadpe',...","{'ancestor': 't1_cr2s3nt', 'author': 'scottevi...",t3_35bc4b
2,Cambion_,"Or more specifically, it is justified in gener...",CMV: The Slut/Stud Inequality Is Generally Jus...,"{'ancestor': 't1_ct27xd0', 'author': 'AdmiralC...","{'ancestor': 't1_ct2avdj', 'author': 'Wolf_Pro...",t3_3d6eeo
3,hldeathmatch,So the most popular non-religious argument aga...,CMV:I think the argument against Zoophillia ba...,"{'ancestor': 't1_csythcw', 'author': 'therapy'...","{'ancestor': 't1_csyxhf1', 'author': 'whattodo...",t3_3cth4w
4,PresidentPikachu,In this post I am going to be very blunt. This...,"CMV: Talking about race, racism, and racial is...","{'ancestor': 't1_cu6c4si', 'author': 'Dopplega...","{'ancestor': 't1_cu6f8lz', 'author': 'PrivateC...",t3_3hcpmz


In [7]:
df = pd.DataFrame()
df['op'] = sample_df['op_text']
df['pos'] = sample_df['positive'].apply(lambda x: x.get('comments')[0].get('body'))
df['neg'] = sample_df['negative'].apply(lambda x: x.get('comments')[0].get('body'))
df['post_id'] = df.index.astype(str)

In [8]:
# SHUFFLE ORDER of 'pos' and 'neg' to create new columns 'response_1' and 'response_2'
import random

def randomize_responses(row):
    if random.random() < 0.5:
        # Keep original order
        row['response_1'] = row['pos']
        row['response_2'] = row['neg']
        row['correct'] = 'r1'
    else:
        # Swap A and B
        row['response_1'] = row['neg']
        row['response_2'] = row['pos']
        row['correct'] = 'r2'
    return row

# apply row-wise
df = df.apply(randomize_responses, axis=1)
df.head()


,op,pos,neg,post_id,response_1,response_2,correct
0,(Edit: My view has been change though there is...,Have you considered the opposite proposition. ...,One of the ideals many people see in a Utopia ...,0,Have you considered the opposite proposition. ...,One of the ideals many people see in a Utopia ...,r1
1,"I'm young, so of course my biggest concern at ...","The major thing you're missing, and really the...","These are good points, and I'm happy to see so...",1,"The major thing you're missing, and really the...","These are good points, and I'm happy to see so...",r1
2,"Or more specifically, it is justified in gener...",The problem with this train of thought is that...,Crime. You are correct in that it's harder for...,2,Crime. You are correct in that it's harder for...,The problem with this train of thought is that...,r2
3,So the most popular non-religious argument aga...,You are saying:\n\n1. We don't require an anim...,&gt; I'm saying that the only way it pet owne...,3,&gt; I'm saying that the only way it pet owne...,You are saying:\n\n1. We don't require an anim...,r2
4,In this post I am going to be very blunt. This...,"While disappointing, I cannot say your view is...",Most people who have some privilege (whether t...,4,Most people who have some privilege (whether t...,"While disappointing, I cannot say your view is...",r2


In [54]:
df.iloc[0]['neg']

'One of the ideals many people see in a Utopia is the abolition of gender. This is the changing of gender roles so much that the two genders become indistinguishable and gender no longer matters. In such a world, transgender people would feel entirely comfortable because there would be no expectations attached to their genitalia. However, with our current society and biology, such a world is a long way off. \n\nTransgender people are for the most part not changing their gender. They were born with a particular sex, yet this does not match their actual internal gender. Coming out is basically admitting to the world that your mental gender differs from your physical sex, in the same way that people who are coming out as gay are not changing their sexual orientation. They were always gay. Transgender people were always the same gender inside, but they simply did not admit it to the world.'

In [ ]:
with open("system_instructions.txt", "r", encoding="utf-8") as f:
    system_prompt = f.read()

print(system_prompt)

In [10]:
import os
from dotenv import load_dotenv
import time
import json
from tqdm import tqdm
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

seen_ids = set()
results = []

# INPUT FOR EACH ROW
def format_user_prompt(row):
    return f"""
Original Post:
{row['op']}

Response 1:
{row['response_1']}

Response 2:
{row['response_2']}
"""

def call_gpt(system_prompt, user_prompt, model="gpt-4o", temperature=0.1, max_tokens=400):
    try:
        response = client.chat.completions.create(
            model=model,
            temperature=temperature,
            max_tokens=max_tokens,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        )
        return response.choices[0].message.content
    except Exception as e:
        print("API error:", e)
        time.sleep(5)
        return None


def parse_response(output):
    try:
        if output.startswith("```json"):
            output = output.removeprefix("```json").removesuffix("```").strip()

        parsed = json.loads(output)

        # flattening JSON
        flat = {}

        # Post-level scores
        flat["malleability"] = parsed["scores"]["original_post"]["malleability"]
        flat["identity_salience"] = parsed["scores"]["original_post"]["identity_salience"]

        # Scores for response_1 and response_2
        for side in ["r1", "r2"]:
            for feature, value in parsed["scores"][side].items():
                flat[f"{side}_{feature}"] = value

        # Final prediction
        flat["more_persuasive"] = parsed["more_persuasive"]

        return flat
    except Exception as e:
        print("Failed to parse JSON:", e)
        return {"gpt_output_raw": output}


In [12]:
# MAIN LOOP

results = []
SAVE_EVERY = 20

for idx, row in tqdm(df.iterrows(), total=len(df)):
    post_id = str(row.get("post_id", idx))
    if post_id in seen_ids:
        continue

    user_prompt = format_user_prompt(row)
    output = call_gpt(system_prompt, user_prompt)

    if output:
        result = parse_response(output)
        result = {"post_id": post_id, **result}
        results.append(result)
        seen_ids.add(post_id)

    if len(results) % SAVE_EVERY == 0:
        temp_df = pd.DataFrame(results)
        temp_df.to_csv("PE_temp.csv", index=False)

    time.sleep(1.2)  # Prevent hitting rate limits


100%|██████████| 500/500 [41:30<00:00,  4.98s/it]  


In [13]:
# MERGE WITH OG DF

results_df = pd.DataFrame(results)
df_PE = df.merge(results_df, on="post_id", how="left")

df_PE.to_csv("PE_final.csv", index=False)

df_PE.head()

,op,pos,neg,post_id,response_1,response_2,correct,malleability,identity_salience,r1_tone_similarity,...,r2_tone_similarity,r2_language_overlap,r2_empathy,r2_external_citation,r2_logic,r2_identity_appeal,r2_politeness,r2_anecdote,r2_concession,more_persuasive
0,(Edit: My view has been change though there is...,Have you considered the opposite proposition. ...,One of the ideals many people see in a Utopia ...,0,Have you considered the opposite proposition. ...,One of the ideals many people see in a Utopia ...,r1,0.8,0.9,0.7,...,0.8,0.6,0.7,0,0.8,0.0,0.7,0.0,0.0,r2
1,"I'm young, so of course my biggest concern at ...","The major thing you're missing, and really the...","These are good points, and I'm happy to see so...",1,"The major thing you're missing, and really the...","These are good points, and I'm happy to see so...",r1,0.8,0.5,0.6,...,0.7,0.5,0.6,0,0.8,0.0,0.7,0.0,0.4,r2
2,"Or more specifically, it is justified in gener...",The problem with this train of thought is that...,Crime. You are correct in that it's harder for...,2,Crime. You are correct in that it's harder for...,The problem with this train of thought is that...,r2,0.2,0.6,0.8,...,0.6,0.5,0.4,0,0.6,0.0,0.6,0.0,0.2,r1
3,So the most popular non-religious argument aga...,You are saying:\n\n1. We don't require an anim...,&gt; I'm saying that the only way it pet owne...,3,&gt; I'm saying that the only way it pet owne...,You are saying:\n\n1. We don't require an anim...,r2,0.5,0.3,0.6,...,0.7,0.6,0.5,0,0.8,0.0,0.6,0.0,0.3,r2
4,In this post I am going to be very blunt. This...,"While disappointing, I cannot say your view is...",Most people who have some privilege (whether t...,4,Most people who have some privilege (whether t...,"While disappointing, I cannot say your view is...",r2,0.5,1.0,0.6,...,0.7,0.5,0.7,1,0.8,0.5,0.7,0.0,0.4,r2


In [ ]:
df_PE.columns

Index(['op', 'pos', 'neg', 'post_id', 'response_1', 'response_2', 'correct',
       'malleability', 'identity_salience', 'r1_tone_similarity',
       'r1_language_overlap', 'r1_empathy', 'r1_external_citation', 'r1_logic',
       'r1_identity_appeal', 'r1_politeness', 'r1_anecdote', 'r1_concession',
       'r2_tone_similarity', 'r2_language_overlap', 'r2_empathy',
       'r2_external_citation', 'r2_logic', 'r2_identity_appeal',
       'r2_politeness', 'r2_anecdote', 'r2_concession', 'more_persuasive',
       'anecdote', 'concession'],
      dtype='object')

In [15]:
(df_PE["more_persuasive"] == df_PE["correct"]).mean()


np.float64(0.638)

In [ ]:
# # Choose the row to test (e.g., first row)
# row = df.iloc[0]
# post_id = str(row.get("post_id", 0))

# # Format the user prompt (original post + responses)
# user_prompt = format_user_prompt(row)

# # Make the API call
# output = call_gpt(system_prompt, user_prompt)

# # Print raw GPT response
# print("GPT Raw Output:\n", output)

# # Parse the JSON
# result = parse_response(output)

# # dictionary unpacking for putting post_id at front
# result = {"post_id": post_id, **result}

# # Convert to DataFrame to preview easily
# test_df = pd.DataFrame([result])
# print("Parsed Output:\n")
# display(test_df)


GPT Raw Output:
 ```json
{
  "more_persuasive": "r2",
  "scores": {
    "original_post": {
      "malleability": 0.8,
      "identity_salience": 0.9
    },
    "r1": {
      "tone_similarity": 0.6,
      "language_overlap": 0.4,
      "empathy": 0.3,
      "external_citation": 0,
      "logic": 0.5,
      "identity_appeal": 0.0,
      "politeness": 0.5,
      "anecdote": 0.0,
      "concession": 0.0
    },
    "r2": {
      "tone_similarity": 0.7,
      "language_overlap": 0.5,
      "empathy": 0.7,
      "external_citation": 0,
      "logic": 0.8,
      "identity_appeal": 0.0,
      "politeness": 0.7,
      "anecdote": 0.0,
      "concession": 0.0
    }
  }
}
```
Parsed Output:



,post_id,malleability,identity_salience,r1_tone_similarity,r1_language_overlap,r1_empathy,r1_external_citation,r1_logic,r1_identity_appeal,r1_politeness,...,r2_tone_similarity,r2_language_overlap,r2_empathy,r2_external_citation,r2_logic,r2_identity_appeal,r2_politeness,r2_anecdote,r2_concession,more_persuasive
0,0,0.8,0.9,0.6,0.4,0.3,0,0.5,0.0,0.5,...,0.7,0.5,0.7,0,0.8,0.0,0.7,0.0,0.0,r2
